In [1]:
import os
import pandas as pd
import numpy as np
from glob import glob
import nibabel as nib
from nilearn.plotting import plot_img, plot_stat_map, view_img, plot_prob_atlas
from nilearn.regions import connected_label_regions
from nilearn.glm.first_level.hemodynamic_models import spm_hrf
from nilearn.image import concat_imgs, mean_img, index_img, smooth_img
from nilearn.glm.first_level import FirstLevelModel
from nilearn.glm import threshold_stats_img
import matplotlib.pyplot as plt
from nilearn.plotting import plot_design_matrix
from nilearn.interfaces.fmriprep import load_confounds_strategy
from nilearn.plotting import plot_roi
from nilearn.maskers import NiftiMapsMasker, NiftiSpheresMasker
from scipy.interpolate import interp1d
import seaborn as sns

# Homemade functions

In [2]:
def Merge(dict1, dict2):

    # create a new dictionary by merging the items of the two dictionaries using the union operator (|)
    merged_dict = dict(dict1.items() | dict2.items())
    
    # return the merged dictionary
    return merged_dict

def merge_dictionaries(dict1, dict2):
    merged_dict = dict1.copy()
    merged_dict.update(dict2)
    return merged_dict

In [3]:
# Define a function to invert scores if needed. 
def invert_score(score):
    if(score == 1):  
        return 4
    elif(score == 2):  
        return 3
    elif(score == 3):  
        return 2
    elif(score == 4):  
        return 1
    else: 
        return "Something went wrong here!"

In [4]:
# Percent signal change .
def get_psc(timecourse):

   # Get number of ROIs and data points in timecourse.
   roi_num = timecourse.shape[1]
   data_length = timecourse.shape[0]

   # Copy timecourse into new array.
   psc_timecourse = np.zeros(timecourse.shape)

   # Warning for empty arrays. 
   if(roi_num ==0):
      print("Watch out, this array is empty!")

   # Loop through every ROI and derive the psc. 
   for id in range(roi_num):

      current_roi_avg = np.mean(timecourse[:, id], axis=0)

      for idx in range(data_length):

         # Formula to get percent signal change -> ((point-avg)/avg)*100.
         psc_timecourse[idx, id] = ((timecourse[idx, id] - current_roi_avg)/ current_roi_avg)*100

   return psc_timecourse

In [5]:
def process_events_data(run_dataframe):

    proccesed_events_df = pd.DataFrame(columns={"Trailer", "Type", "Onset", "Duration", "Offset", "W_score", "A_score", "F_score"}) 

    # Initial fixation 12 sec (TR=6).
    in_fix = 12

    # Time it take subjects to complete questionnaire 12 sec (TR=6). 
    questionnaire_duration = 12

    # All trailers last 30 sec (TR=15). 
    trailer_duration = 30

    # Initialize this variable, though it will change through each iteration of the loop.
    trailer_onset = 0

    # Run a for loop for each row in the df. 
    for id in range(run_dataframe.shape[0]):

        # Get trailer label and separate them accroding to their type. 
        trailer_name = run_dataframe["label"][id]
        trailer_type = "Horror" if "h" in run_dataframe["label"][id] else "Comedy"

        # This onsets don't work, so I need to re-calculate them
        traile_iti = run_dataframe["trial_ITI"][id]
        
        # For the first run add the initial fixation time to the calculation of the first trailer onset. 
        # After the first run, calculate onset by adding previous traile onset, questionnaire duration, and trial iti.
        if (id == 0):
            trailer_onset += in_fix
        else:
            trailer_onset += trailer_duration + questionnaire_duration + traile_iti

        # Calculate trailer onset. 
        trailer_offset = trailer_onset + 30 

        """ 
        For the questionnaire scores, as I understood it. If they were not inverted (["scale_flip"] == 0), then 
        the lower the score the stronger the response. If they were inverted (["scale_flip"] == 1), the higher the 
        score the stronger the response.
        """
        trailer_watch_score = run_dataframe["exp_Watch.keys"][id].astype(int)
        trailer_arousal_score = run_dataframe["exp_Arousal.keys"][id].astype(int)
        trailer_feel_score = run_dataframe["exp_Feel.keys"][id].astype(int)

        # Check if scaled was flipped and put scores on the same scale. 
        # For me, the most intuitive is that the higher the score, the stronger the response. 
        if(run_dataframe["scale_flip"][id] == 0):
            trailer_watch_score = run_dataframe["exp_Watch.keys"].apply(invert_score)[id].astype(int)
            trailer_arousal_score = run_dataframe["exp_Arousal.keys"].apply(invert_score)[id].astype(int)
            trailer_feel_score = run_dataframe["exp_Feel.keys"].apply(invert_score)[id].astype(int)
        
        # Place processed data on list, add list to new dataframe, and concat to main dataframe. 
        current_row_data = [[trailer_name, trailer_type, trailer_onset, trailer_duration, trailer_offset, trailer_watch_score, trailer_arousal_score, trailer_feel_score]]
        current_row = pd.DataFrame(data=current_row_data, columns=["Trailer", "Type", "Onset", "Duration", "Offset", "W_score", "A_score", "F_score"]) 
        proccesed_events_df = pd.concat([proccesed_events_df, current_row], ignore_index=True)
        proccesed_events_df = proccesed_events_df[["Trailer", "Type", "Onset", "Offset", "Duration", "W_score", "A_score", "F_score"]]

    return proccesed_events_df

In [6]:
# Open a datasets directory. 
fd = os.open("/Users/luisalvarez/Documents/Datasets", os.O_RDONLY)

# Use os.fchdir() method to change the current dir/folder.
os.fchdir(fd)

# Safe check- Print current working directory
print("Current working dir : %s" % os.getcwd())

Current working dir : /Users/luisalvarez/Documents/Datasets


In [7]:

def getNAcc_timecourse(participant_num): 

    # Add code to flag if something that is not a number is passed. 

    ## 1) Load data.
    # Load relevant files for participant
    sub_run1_func_path = "MovieData_BIDS_preproc/sub-" + participant_num + "/func/sub-" + participant_num + "_task-movie_run-01_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz"
    sub_run2_func_path = "MovieData_BIDS_preproc/sub-" + participant_num + "/func/sub-" + participant_num + "_task-movie_run-02_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz"

    sub_run1_mask_path = "MovieData_BIDS_preproc/sub-" + participant_num + "/func/sub-" + participant_num + "_task-movie_run-01_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz"
    sub_run2_mask_path = "MovieData_BIDS_preproc/sub-" + participant_num + "/func/sub-" + participant_num + "_task-movie_run-02_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz"

    sub_run1_events_path = "MovieData_BIDS_raw/sub-" + participant_num + "/func/sub-" + participant_num + "_task-movie_run-01_events.csv"
    sub_run2_events_path = "MovieData_BIDS_raw/sub-" + participant_num + "/func/sub-" + participant_num + "_task-movie_run-02_events.csv"

    sub_run1_confounds_path = "MovieData_BIDS_preproc/sub-" + participant_num + "/func/sub-" + participant_num + "_task-movie_run-01_desc-confounds_timeseries.tsv"
    sub_run2_confounds_path = "MovieData_BIDS_preproc/sub-" + participant_num + "/func/sub-" + participant_num + "_task-movie_run-01_desc-confounds_timeseries.tsv"

    # Calculate relevant parameters for GLM and ROI time-course analysis.
    func_file = nib.load(sub_run1_func_path)
    func_data = func_file.get_fdata()
    n_vols = func_data.shape[3]
    TR = 2
    n_timepoints = n_vols*TR

    # Load raw events files. 
    sub_run1_events_df = pd.read_csv(sub_run1_events_path, index_col=0)
    sub_run2_events_df = pd.read_csv(sub_run2_events_path, index_col=0)

    ## 2) Process files. 
    # Process event files. 
    sub_run1_p_events = process_events_data(sub_run1_events_df)
    sub_run2_p_events = process_events_data(sub_run2_events_df)

    # Down-sample time onsets to get vol onsets. 
    # Create array from 0 to 'n_timepoints' in steps of 1.
    time_scale = np.arange(0, n_timepoints, 1)  

    # Create array from 0 to 'n_timepoints' in steps of 2.
    vol_scale = np.arange(0, n_timepoints, TR)  

    # Get the labels of each trailer for each run. 
    run1_trailer_labels = sub_run1_p_events["Trailer"].tolist()
    run2_trailer_labels = sub_run2_p_events["Trailer"].tolist()

    # Create dictionary variable to store arrays with onset values for each trailer. 
    run1_onsets = {}
    run2_onsets = {}

    # Create a dictionary with all the onsets for each trailer in each run. 
    for id in range(len(run1_trailer_labels)):

        # Create array of zeros.
        run1_trailer_onsets = np.zeros(n_timepoints)
        run2_trailer_onsets = np.zeros(n_timepoints)

        # Get onset time. 
        run1_current_trailer_onset = sub_run1_p_events["Onset"][id]
        run2_current_trailer_onset = sub_run2_p_events["Onset"][id]

        # Assign 1 to such onset all the way til the end of the trailer (30 sec) in the array of zeros.
        # Adjust for lag: add 4 seconds at the onset and offset
        run1_trailer_onsets[int(run1_current_trailer_onset + 4):int(run1_current_trailer_onset)+ 30 + 4] = 1
        run2_trailer_onsets[int(run2_current_trailer_onset + 4):int(run2_current_trailer_onset)+ 30 + 4] = 1

        # Create resampler objects for each trailer/run of reward.
        run1_resampler = interp1d(time_scale, run1_trailer_onsets)
        run2_resampler = interp1d(time_scale, run2_trailer_onsets)

        # Create downsampled arrays for each trailer. 
        # Note this vol arrays are half the length than the time arrays.
        run1_trailer_vol_onsets = run1_resampler(vol_scale)
        run2_trailer_vol_onsets = run2_resampler(vol_scale)

        # Append/store the downsampled volumes arrays to each dictionary.
        # I'm doing it this way, so the code is more interpretable
        run1_onsets[run1_trailer_labels[id]] = run1_trailer_vol_onsets
        run2_onsets[run2_trailer_labels[id]] = run2_trailer_vol_onsets

    ## 3) Load confound data. 
    sub_run1_confounds_df = pd.read_csv(sub_run1_confounds_path, sep='\t')
    sub_run2_confounds_df = pd.read_csv(sub_run2_confounds_path, sep='\t')
    default_confounds = ['trans_y', 'trans_x', 'trans_z', 'rot_y', 'rot_x', 'rot_z',
                        "white_matter", "csf", "csf_wm",  "framewise_displacement", "dvars"]

    # Add confound columns if they contain 'motion' in the title. 
    sub_run1_motion_confounds = [i for i in sub_run1_confounds_df.columns if "state" in i] #"motion"
    sub_run2_motion_confounds = [i for i in sub_run2_confounds_df.columns if "state" in i] 
    sub_run1_filtered_confounds_df = sub_run1_confounds_df[default_confounds + sub_run1_motion_confounds[0:3]]
    sub_run2_filtered_confounds_df = sub_run2_confounds_df[default_confounds + sub_run2_motion_confounds[0:3]]

    # Change NaNs to 0s. 
    sub_run1_filtered_confounds_df = sub_run1_filtered_confounds_df.fillna(0) 
    sub_run2_filtered_confounds_df = sub_run2_filtered_confounds_df.fillna(0) 

    ## 4) Apply mask to func data. 
    # Applying a Gaussian filter with a 4mm kernel
    sub_run1_func_smooth = smooth_img(sub_run1_func_path, 4)
    sub_run2_func_smooth = smooth_img(sub_run2_func_path, 4)

    masker_sNAcc_r1 = NiftiSpheresMasker(
        seeds=[(10, 12, -2), (-10, 12, -2)],  # right, left
        radius=8, 
        mask_img=sub_run1_mask_path,
        standardize=False, 
        t_r=2,
        standardize_confounds=True, 
        high_pass=0.002,
        low_pass=0.1 # from 1.0
        )

    masker_sNAcc_r2 = NiftiSpheresMasker(
        seeds=[(10, 12, -2), (-10, 12, -2)],  # right, left
        radius=8, 
        mask_img=sub_run2_mask_path,
        standardize=False, 
        t_r=2,
        standardize_confounds=True,
        high_pass=0.002,
        low_pass=0.1 # from 1.0
        )

    # Mask the func data and get a time series for the ROI. 
    # Note this is similar to fitting the GLM, but without the event files.
    sub_r1_bNAcc = masker_sNAcc_r1.fit_transform(sub_run1_func_smooth, confounds=sub_run1_filtered_confounds_df)
    sub_r2_bNAcc = masker_sNAcc_r2.fit_transform(sub_run2_func_smooth, confounds=sub_run2_filtered_confounds_df)

    # Apply function to get the percent signal change from each ROI timecourse. 
    sub_r1_bNAcc_psc = get_psc(sub_r1_bNAcc)
    sub_r2_bNAcc_psc = get_psc(sub_r2_bNAcc)

    ## 5) Get the timecourses from each movie trailer. 
    # Create dictionary variable to store arrays with time series arrays for each trailer. 
    run1_timeseries = {}
    run2_timeseries = {}

    # Get the trailers presented in each run. 
    r1_keys = list(run1_onsets.keys())
    r2_keys = list(run2_onsets.keys())

    # Loop through each traile and get its corresponding ROI timecourse
    # from bilateral, left, and right, NAcc.
    for id in range(len(r1_keys)):

        run1_timeseries[r1_keys[id]] = {
            "Bilateral_NAcc": np.mean(sub_r1_bNAcc_psc[run1_onsets[r1_keys[id]].astype(bool)][:, :], axis=1),
            "Left_NAcc": sub_r1_bNAcc_psc[run1_onsets[r1_keys[id]].astype(bool)][:, 1],
            "Right_NAcc": sub_r1_bNAcc_psc[run1_onsets[r1_keys[id]].astype(bool)][:, 0]}
    
        run2_timeseries[r2_keys[id]] = {
            "Bilateral_NAcc": np.mean(sub_r2_bNAcc_psc[run2_onsets[r2_keys[id]].astype(bool)][:, :], axis=1),
            "Left_NAcc": sub_r2_bNAcc_psc[run2_onsets[r2_keys[id]].astype(bool)][:, 1],
            "Right_NAcc": sub_r2_bNAcc_psc[run2_onsets[r2_keys[id]].astype(bool)][:, 0]}

    all_timeseries = merge_dictionaries(run1_timeseries, run2_timeseries)

    return all_timeseries



In [8]:
sub01_NAcc_timecourse = getNAcc_timecourse("01")
sub02_NAcc_timecourse = getNAcc_timecourse("02")
sub03_NAcc_timecourse = getNAcc_timecourse("03")
sub04_NAcc_timecourse = getNAcc_timecourse("04")
sub05_NAcc_timecourse = getNAcc_timecourse("05")
